# Category Theory for Programmers

## Chapter 2: Types and Functions

### Challenge 5
There are $|\text{Bool}|^{|\text{Bool}|} = 2^2 = 4$ functions from `Bool` to `Bool`.

In [ ]:
f1 :: Bool -> Bool
f1 = id

f2 :: Bool -> Bool
f2 = not

f3 :: Bool -> Bool
f3 = const False

f4 :: Bool -> Bool
f4 = const True

## Chapter 3: Categories Great and Small

### Challenge 3

```haskell
import Data.Monoid

instance Monoid Bool where
    mempty = True
    mappend = (&&)

instance Monoid Bool where
    mempty = False
    mappend = (||)
```

### Challenge 5

In [ ]:
import Data.Monoid
import Data.Semigroup

newtype Z3 = Z3 Int
    deriving Show

instance Semigroup Z3 where
    (Z3 x) <> (Z3 y) = Z3 ((x + y) `mod` 3)

instance Monoid Z3 where
    mempty = Z3 0

In [ ]:
foldMap Z3 [2, 1, 2]

## Chapter 6: Simple Algebraic Data Types

### Challenge 1
Show the isomorphism between `Maybe a` and `Either () a`.

In [ ]:
maybeToEither :: Maybe a -> Either () a
maybeToEither Nothing = Left ()
maybeToEither (Just x) = Right x

eitherToMaybe :: Either () a -> Maybe a
eitherToMaybe (Left _) = Nothing
eitherToMaybe (Right x) = Just x

In [ ]:
eitherToMaybe $ maybeToEither Nothing

In [ ]:
eitherToMaybe $ maybeToEither $ Just 42

### Challenge 4
Define new shape `Square` and extend `area` and `circ`.

In [ ]:
data Shape = Circle Float | Rect Float Float | Square Float

area :: Shape -> Float
area (Circle r) = pi * r^2
area (Rect d h) = d * h
area (Square s) = s^2

circ :: Shape -> Float
circ (Circle r) = 2 * pi * r
circ (Rect d h) = 2 * (d + h)
circ (Square s) = 4 * s

### Challenge 5
Show that $a + a = 2 \times a$ up to isomorphism where $a \times b$ is `(a, b)` and $a + b$ is `Either a b` ($2$ in this setup corresponds to `Bool`).

The definition below works up to isomorphism in the choice between
 1. `Left -> False` vs `Left -> True`
 1. `(Bool, a)` vs `(a, Bool)`

In [ ]:
toFlagged :: Either a a -> (Bool, a)
toFlagged (Left x) = (False, x)
toFlagged (Right x) = (True, x)

fromFlagged :: (Bool, a) -> Either a a
fromFlagged (False, x) = Left x
fromFlagged (True, x) = Right x

## Chapter 7: Functors

### Challenge 1
Can the following definition for `Maybe` be made into a `Functor`?
```haskell
fmap :: (a -> b) -> Maybe a -> Maybe b
fmap _ _ = Nothing
```

#### Id Law: $\text{fmap}_f \text{id}_a = \text{id}_{f a}$

- Case `Nothing`:
```haskell
fmap id Nothing
= Nothing
= id Nothing
```
- Case `Just` is not satisfied!
```haskell
fmap id (Just x)
= Nothing
/= Just x
= id (Just x)
```

### Challenge 2
Prove the `Functor` laws for the *Reader* functor:
```haskell
instance Functor ((->) r) where
    -- fmap :: (a -> b) -> (r -> a) -> (r -> b)
    fmap f g = f . g
```

#### Id Law: `fmap id = id`
```haskell
fmap id f
= id . f
= f
```
So `fmap id` has the same effect on `f` as if `f` was just alone, i.e. the same as `id` has. Therefore `fmap id = id`. 

#### Composition Law: `fmap (g . f) = fmap g . fmap f`
Case `Just` is not satisfied!
```haskell
fmap (g . f) h
= (g . f) . h
= g . f . h
= g . (f . h)
= fmap g (f . h)
= fmap g (fmap f h)
= (fmap g . fmap f) h
```

### Challenge 4
Prove the functor laws for the list functor.

```haskell
instance Functor [] where
    -- fmap :: (a -> b) -> [a] -> [b]
    fmap _ [] = []
    fmap f (x:xs) = f x : (fmap f xs)
```

#### Id Law: `fmap id = id`
Base case `[]`:
```haskell
fmap id []
= []
```
i.e. `fmap id` acts as `id` on `[]`.

For the inductive step, let's assume that the relation `fmap id xs = xs` holds for some list `xs`. Then
```haskell
fmap id (x:xs)
= {- applying the definition of fmap -}
  id x : (fmap id xs)
= {- applying id on x -}
  x : (fmap id xs)
= {- induction hypothesis -}
  x : xs
```
proves that `fmap id` acts on `(x:xs)` just like `id`.

#### Composition Law: `fmap (g . f) = fmap g . fmap f`
Base case `[]`:
```haskell
fmap (g . f) []
= []
= fmap g []
= fmap g (fmap f [])
= (fmap g . fmap f) []
```

For the inductive step, let's assume that the relation `fmap (g . f) xs = (fmap g . fmap f) xs` holds for some list `xs`. Then
```haskell
fmap (g . f) (x:xs)
= {- applying the definition of fmap -}
  (g . f) x : (fmap (g . f) xs)
= {- applying (g . f) on x -}
  g (f x) : (fmap (g . f) xs)
= {- induction hypothesis -}
  g (f x) : ((fmap g . fmap f) xs)
= {- writing down (fmap g . fmap f) explicitly -}
  g (f x) : (fmap g (fmap f xs))
= {- factoring out g mapping the head (f x) and the tail (fmap f xs) as (fmap g) over the cons operator -}
  fmap g (f x : (fmap f xs))
= {- repeat previous step for f -}
  fmap g (fmap f (x:xs))
= {- rewrite double fmap application as a composition (fmap g . fmap f) -}
  (fmap g . fmap f) (x:xs)
```
proves that `fmap (g . f)` acts on `(x:xs)` just like `(fmap g . fmap f)`.

## Chapter 8: Functoriality

### Challenge 1
Show that `Pair` (defined below) is a *bifunctor*.

In [ ]:
import Data.Bifunctor

data Pair a b = Pair a b
    deriving Show

instance Bifunctor Pair where
    -- default: `bimap f g = first g . second h`
    -- bimap :: (a -> c) -> (b -> d) -> Pair a b -> Pair c d
    bimap g h (Pair x y) = Pair (g x) (h y)
    
    -- default: `first g = bimap g id`
    -- first :: (a -> c) -> Pair a b -> Pair c b
    first g (Pair x y) = Pair (g x) y
    
    -- default: `second = bimap id`
    -- second :: (b -> d) -> Pair a b -> Pair a d
    second h (Pair x y) = Pair x (h y)

In [ ]:
bimap (+1) (^2) (Pair 41 3)

In [ ]:
first (+1) (Pair 41 3)

In [ ]:
second (^2) (Pair 41 3)

Additionally, let's show that the definitions above are compatible with mentioned default implementations.

#### `bimap`
```haskell
bimap g h (Pair x y)
= {- default definition -}
  (first g . second h) (Pair x y)
= {- definition of composition -}
  first g (second h (Pair x y))
= {- apply definition for second and eliminate id on x -}
  first g (Pair x (h y))
= {- apply definition for first and eliminate id on (h y) -}
  Pair (g x) (h y)
= {- custom definition -}
  bimap g h (Pair x y)
```

#### `first`
```haskell
first g (Pair x y)
= {- default definition -}
  bimap g id (Pair x y)
= {- definition of bimap -}
  Pair (g x) (id y)
= {- apply id to y -}
  Pair (g x) y
= {- custom definition -}
  first g (Pair x y)
```

#### `second`
```haskell
second h (Pair x y)
= {- default definition -}
  bimap id h (Pair x y)
= {- definition of bimap -}
  Pair (id x) (h y)
= {- apply id to x -}
  Pair x (h y)
= {- custom definition -}
  second h (Pair x y)
```

### Challenge 2
Show the isomorphism between the standard definition of `Maybe` and desugared custom definition `Maybe'`.

```haskell
data Maybe a = Nothing | Just a
```

In [ ]:
import Data.Functor.Const
import Data.Functor.Identity

type Maybe' a = Either (Const () a) (Identity a)

preludeToPrime :: Maybe a -> Maybe' a
preludeToPrime Nothing = Left (Const ())
preludeToPrime (Just x) = Right (Identity x)

primeToPrelude :: Maybe' a -> Maybe a
primeToPrelude (Left _) = Nothing
primeToPrelude (Right (Identity x)) = Just x

Additionally, let's shorten `preludeToPrime` to `f` and `primeToPrelude` to `f'` and show that these are inverses, i.e. `f' . f = id`.

#### Case `Nothing` / `Const () a`
```haskell
(f' . f) Nothing
= {- definition of composition -}
  f' (f Nothing)
= {- apply f (preludeToPrime) -}
  f' (Left (Const ()))
= {- apply f' (primeToPrelude) -}
  Nothing
```
So `(f' . f)` acts on `Nothing` just as `id` does.

#### Case `Just a` / `Identity a`
```haskell
(f' . f) (Just x)
= {- definition of composition -}
  f' (f (Just x))
= {- apply f (preludeToPrime) -}
  f' (Right (Identity x)))
= {- apply f' (primeToPrelude) -}
  Just x
```
Again, `(f' . f)` is equivalent to `id` when applied to `Just x`.

### Challenge 3
Give an instance of `Bifunctor` for `PreList` which is defined below.
*Note: `PreList` can be viewed as a precursor to `List`.*

In [ ]:
data PreList a b = Nil | Cons a b

instance Bifunctor PreList where
    -- bimap :: (a -> b) -> (c -> d) -> PreList a c -> PreList b d
    bimap _ _ Nil = Nil
    bimap g h (Cons x y) = Cons (g x) (h y)

### Challenge 4
Give `Bifunctor` instances for following types.

In [ ]:
newtype K2 c a b = K2 c

instance Bifunctor (K2 x) where
    -- bimap :: (a -> b) -> (c -> d) -> (K2 x) a c -> (K2 x) b d
    bimap _ _ (K2 x) = K2 x

newtype Fst a b = Fst a

instance Bifunctor Fst where
    -- bimap :: (a -> b) -> (c -> d) -> Fst a c -> Fst b d
    bimap g _ (Fst x) = Fst (g x)

newtype Snd a b = Snd b

instance Bifunctor Snd where
    -- bimap :: (a -> b) -> (c -> d) -> Snd a c -> Snd b d
    bimap _ h (Snd x) = Snd (h x)

## Chapter 10: Natural Transformations

### Challenge 1
Define natural transformation between `Maybe a` and `[a]` functors.

In [ ]:
toList :: Maybe a -> [a]
toList Nothing = []
toList (Just x) = [x]

In [ ]:
toList $ Just 42

In [ ]:
toList Nothing

Furthermore, let's prove the *naturality condition* for `toList` defined above:

#### `Nothing`
```haskell
(fmap f . toList) Nothing
= {- definition of composition -}
  fmap f (toList Nothing)
= {- definition of toList -}
  fmap f []
= {- definition of fmap for the list functor -}
  []
= {- definition of toList -}
  toList Nothing
= {- definition of fmap for the Maybe functor -}
  toList (fmap f Nothing)
= {- definition of composition -}
  (toList . fmap f) Nothing
```
which proves the naturality condition for the first case (`Nothing`).

#### `Just x`
```haskell
(fmap f . toList) (Just x)
= {- definition of composition -}
  fmap f (toList (Just x))
= {- definition of toList -}
  fmap f [x]
= {- definition of fmap for the list functor -}
  [f x]
= {- definition of toList -}
  toList (Just (f x))
= {- definition of fmap for the Maybe functor -}
  toList (fmap f (Just x))
= {- definition of composition -}
  (toList . fmap f) (Just x)
```
This concludes the proof of the naturality condition `fmap f . toList = toList . fmap f`.

### Challenge 2
Define two natural transformations between `Reader () a` and `[a]` functors.

In [ ]:
type Reader r a = r -> a

alpha :: Reader () a -> [a]
alpha r = [r ()]

beta :: Reader () a -> [a]
beta r = r () : beta r

### Challenge 3
Define two natural transformations between `Reader Bool a` and `Maybe a` functors.

In [ ]:
alpha :: Reader Bool a -> Maybe a
alpha r = Just (r True)

beta :: Reader Bool a -> Maybe a
beta r = Just (r False)

## Chapter 12: Limits and Colimits

### Challenge 2
Show that the limit of the identity functor $\mathbf{Id}: C \to C$ is the initial object.

```haskell
newtype Id a = Id a

instance Functor Id where
    -- fmap :: (a -> b) -> f a -> f b
    fmap f (Id x) = Id (f x)
```

Since $\mathbf{Id}$ just "copies" $C$ as the diagram, $\mathrm{Lim}\:\mathbf{Id}$ will have a unique morphism any object in $C$. This morphism is unique up to unique isomorphism with some other candidate $\Delta c$. I.e. $C(\Delta_c, \mathrm{Lim}\:\mathbf{Id}) \simeq Nat(\Delta_c, \mathbf{Id})$. This description corresponds to the definition of the *initial object* (here $\mathrm{Lim}\;\mathbf{Id}$).

### Challenge 3 *[TODO]*
Subsets $A \subseteq B$ of given set $B$ form a category:
 - objects are subsets of $B$
 - there's a morphism $A \to B$ iff $A \subseteq B$
 - identity, composition and associativity of morphisms follow from set theory

What is the *pullback*, *pushout* and *initial* and *terminal* objects?

### Challenge 4
See the defintion an examples of a *coequalizer*
 - either on [Wikipedia](https://en.wikipedia.org/wiki/Coequalizer)
 - or on the [nLab](https://ncatlab.org/nlab/show/coequalizer)

### Challenge 5 *[TODO]*
Show that in a category with a termianl object, a pullback towards the terminal object is a product.

### Challenge 6 *[TODO]*
Show that in a category with an initial object, a pullout from the initial object is a coproduct.

## Chapter 13: Free Monoids

### Challenge 1
Show that an isomorphism between monoids that preserves multiplication must automatically preserve unit.

Let's assume there's an isomorphism $h$ preserving multiplication: $h(a \bullet b) = h(a) \bullet h(b)$. Therefore for a unit $e$ it must hold that $h(a) \bullet h(e) = h(a \bullet e) = h(a)$ (and similarly for the right unit).

Let's assume there could be a "true" unit $e' \neq h(e)$ outside the image of $h$ for the target monoid. Then $h^{-1}(e') = x \neq e$ but also
$$
h(a \bullet e) = h(a) \bullet h(e) = h(a) = h(a) \bullet e' = h(a \bullet h^{-1}(e')) = h(a \bullet x)
$$
where in the last term $x \neq e$. This is in contradiction with the assumption that $h$ is an isomorphism. $\blacksquare$

### Challenge 2
Consider monoid homomorphism `h` from `([Int], (++))` to `(Int, (*))`.

**What is `h []`?**

`h [] = 1` because `xs ++ [] = xs = [] ++ xs` and so does `1` for `(*)`.

**Assuming `h [x] = x`, what's `h [1, 2, 3, 4]`?**
```haskell
h [1, 2, 3, 4]
= {- definition of multiplication in Monoid [Int] -}
  h ([1] ++ [2] ++ [3] ++ [4])
= {- definition of multiplication in Monoid Int and the fact that h preserves monoidal multiplication -}
  h [1] * h [2] * h [3] * h[4]
= {- defintion of h -}
  1 * 2 * 3 * 4
= {- multiplication in Monoid Int -}
  24
```

**How many different lists map to 12?**

The number of different lists `xs` is the count of `xs` solving `prod xs = 12`.

`[12]`, `[2, 6]`, `[2, 2, 3]`, `[3, 4]` all map to 12 under `h`. Additionally there are permutations of these and disregarding 1 as the unit for `(*)` (i.e. the fact that `h [1, 1, 1, 1, 2, 3] = 12 = h [2, 3]`).

### Challenge 3 *[TODO]*
What is the *free monoid* generated by a one-element set? What is it isomorphic to?

## Chapter 14: Representable Functors

### Challenge 1
Show that the *hom-functors* map identity morphisms in $C$ to corresponding identity functions in $\mathrm{Set}$.

Let's start with morphisms $h \in C(a, x)$ and $f: x \to y$ such that $f \circ h \in C(a, y)$ for arbitrary objects $a, x, y \in C$. Than if we assume $x = y$ (i.e. degenerate the commuting triangle), $f = id_x$.

But then the hom-functor maps this as $C(a, id)h = h \circ id = h$ which shows that it is the identity function in $\mathrm{Set}$.

## Challenge 2
Show that the `Maybe` functor is not *representable*.

In order to show that `Maybe` is representable, there must be an invertible natural transformation, i.e. two morphisms such that $\alpha \circ \beta = id = \beta \circ \alpha$:
```haskell
-- Note: think of this as `tabulate`
alpha :: (a -> x) -> Maybe x

-- Note: think of this as `index`
beta :: Maybe x -> (a -> x)
```

The problem with `Maybe`, and with `Nothing` in particular, is that there is no `beta Nothing` that would produce an indexation function yielding `x` that would not break the invertibility of the natural transformation.

```haskell
alpha f = Just . f

beta (Maybe x) = \_ -> x
beta Nothing = undefined
```

### Challenge 3
Is the `Reader` functor *representable*? Yes, there's a trivial isomorphism to `r -> a`.
```haskell
type Reader r a = r -> a

-- From `adjuctions` package sources
instance Representable ((->) e) where
  type Rep ((->) e) = e
  index = id
  tabulate = id
```

### Challenge 4
Using `Stream` representation (given below), memoize a function that squares its argument.

In [ ]:
-- from package `adjuctions`
-- import Data.Functor.Rep
-- or from package `representable-functors`
-- import Data.Functor.Representable

{-# LANGUAGE TypeFamilies #-}
{-# LANGUAGE FlexibleContexts #-}

class Representable f where
    type Rep f :: *
    tabulate :: (Rep f -> x) -> f x
    index :: f x -> Rep f -> x

data Stream x = Cons x (Stream x)

instance Representable Stream where
    type Rep Stream = Integer
    tabulate f = Cons (f 0) (tabulate (f . (+1)))
    index (Cons b bs) n
        | n < 0 = error "key in index must be a non-negative integer"
        | n == 0 = b
        | otherwise = index bs (n - 1)

In [ ]:
square :: Integer -> Integer
square = index squares
    where
        -- This type annotation is needed.
        -- For some reason the second argument of `index` would not otherwise match correctly.
        squares :: Stream Integer
        squares = tabulate (^2)

In [ ]:
square 3

### Challenge 6
Implement `Representable` for the type `Pair a = Pair a a`.

In [ ]:
data Pair a = Pair a a

instance Representable Pair where
    type Rep Pair = Bool
    tabulate f = Pair (f True) (f False)
    index (Pair x y) first = if first then x else y